In [20]:
import numpy as np
from numpy.linalg import norm
from legacy import main as legacy
from legacy import legacy_adapter as adapter

In [21]:
def lbfgs(grad, generations, points, eps, max_iter):
    if len(points) < generations + 1:
        raise AttributeError("The length of s or y is too small")
    
    E = np.eye(generations)
    point = points[-1]
    
    s = [points[i] - points[i - 1] for i in range(1, generations + 1)]
    y = [grad(points[i]) - grad(points[i - 1]) for i in range(1, generations + 1)]
    rho = [1 / y[i] @ s[i] for i in range(generations)]
    
    for ki in range(0, max_iter):
        q = grad(point)
        g = q
        
        if norm(q) < eps:
            break
        
        alpha = np.zeros(generations)
        for it in range(0, generations):
            i = generations - (it + 1)
            alpha[i] = rho[i] * (s[i] @ q)
            q = q - alpha[i] * y[i]
        
        gamma = (s[-1] @ y[-1]) / (y[-1] @ y[-1])
        hess = gamma * E
        
        z = hess @ q
        
        betta = np.zeros(generations)
        for i in range(0, generations):
            betta[i] = rho[i] * (y[i] @ z)
            z = z + s[i] * (alpha[i] - betta[i])
        
        # z = -H_k @ g_k
        point += z
        
        s = s[1:]
        s.append(z)
        
        y = y[1:]
        y.append(grad(point) - g)
        
        rho = rho[1:]
        rho.append(1 / y[-1] @ s[-1])
        
    return point

In [21]:
def f(point):
    x, y = point[0], point[1]
    return x ** 2 - x * y + y ** 2 + 9 * x - 6 * y + 20

def grad(point):
    x, y = point[0], point[1]
    return np.array([[2 * x - y + 9], [-x + 2 * y - 6]])

Для нахождения первых m s и y хотелось бы использовать какие-то алгоритмы, которые могут хоть как-то приближают нас к минимуму, для этого можно
1. Воспользоваться многочисленными вариантами градиентного спуска
2. Запустить BFGS, но это будет не выгодно, т.к. мы изначально хотим избавиться от хранения большой матрицы, то есть мы не решаем проблему, хотя в дальнейшем нам не нужно будет каждый раз высчитывать гигантскую матрицу
3. Powell dog leg -- звучит как что-то не то, что хотелось бы использовать
4. G-N не та задача

То есть самым лучшим вариантом остаётся градиентный спуск и его вариации